In [1]:
import json
from math import comb

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
from datetime import datetime, timedelta
from pathlib import Path

directory = Path("data/Biketown Raw Data")
filtered_dfs = []
column_mapping = {
    'Start_Latitude': 'StartLatitude',
    'Start_Longitude': 'StartLongitude',
    'End_Latitude': 'EndLatitude',
    'End_Longitude': 'EndLongitude',
    'Distance_Miles_': 'DistanceMiles',
    'Distance_Miles': 'DistanceMiles',
}

for file_path in directory.rglob('*'):
    year, month = map(int, file_path.stem.split('_'))
    df = pd.read_csv(file_path)
    
    df.rename(columns=column_mapping, inplace=True)
    
    # combined_df['StartDateTime'] = pd.to_datetime(combined_df['StartDate'] + ' ' + combined_df['StartTime'])
    # combined_df['EndDateTime'] = pd.to_datetime(combined_df['EndDate'] + ' ' + combined_df['EndTime'])
    
    df['StartDate'] = pd.to_datetime(df['StartDate'] + ' ' + df['StartTime'])
    df['EndDate'] = pd.to_datetime(df['EndDate'] + ' ' + df['EndTime'])
    # df['EndDate'] = pd.to_datetime(df['EndDate'])
    
    nan_duration_mask = df['Duration'].isna()
    calculated_duration = (df['EndDate'] - df['StartDate'])
    df.loc[nan_duration_mask, 'Duration'] = calculated_duration[nan_duration_mask]
    df.loc[~nan_duration_mask, 'Duration'] = pd.to_timedelta(df.loc[~nan_duration_mask, 'Duration'])
    
    start_of_month = datetime(year, month, 1)
    end_of_month = datetime(year, month + 1, 1) - timedelta(seconds=1) if month < 12 else datetime(
        year + 1, 1, 1
        ) - timedelta(
        seconds=1
    )
    
    df_filtered = df[
        (df['StartDate'] >= start_of_month - timedelta(days=1)) & (df['EndDate'] <= end_of_month + timedelta(days=1))]
    filtered_dfs.append(df_filtered)

og_combined_df = pd.concat(filtered_dfs, ignore_index=True)

/var/folders/_z/q4gsch7n2jn4pky14jnx_2p80000gn/T/ipykernel_5409/3363474797.py:17: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/_z/q4gsch7n2jn4pky14jnx_2p80000gn/T/ipykernel_5409/3363474797.py:17: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/_z/q4gsch7n2jn4pky14jnx_2p80000gn/T/ipykernel_5409/3363474797.py:17: DtypeWarning: Columns (12,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [108]:
coords = pd.concat(
    [
        og_combined_df[['StartLatitude', 'StartLongitude']],
        og_combined_df[['EndLatitude', 'EndLongitude']].rename(
            columns={'EndLatitude': 'StartLatitude', 'EndLongitude': 'StartLongitude'}
        )
    ]
)

original_unique_coords = len(coords.drop_duplicates())

for i in range (9):
    rounded_unique_coords = len(coords.round(i).drop_duplicates())
    print(f"{i}: {rounded_unique_coords/original_unique_coords} ||| {rounded_unique_coords} / {original_unique_coords}")


0: 0.002386634844868735 ||| 11 / 4609
1: 0.00694293773052723 ||| 32 / 4609
2: 0.08396615317856368 ||| 387 / 4609
3: 0.7114341505749621 ||| 3279 / 4609
4: 0.973096116294207 ||| 4485 / 4609
5: 0.9965285311347364 ||| 4593 / 4609
6: 0.9989151659796052 ||| 4604 / 4609
7: 1.0 ||| 4609 / 4609
8: 1.0 ||| 4609 / 4609


In [161]:
combined_df = og_combined_df.copy()
combined_df = combined_df[combined_df['Duration'] > timedelta(days=0)]
combined_df = combined_df[(30 >= combined_df['DistanceMiles']) & (combined_df['DistanceMiles'] > 0)]
combined_df = combined_df[(combined_df['Duration'] <= timedelta(days=1)) | (combined_df['DistanceMiles'] >= 1)]
combined_df['Duration'] = pd.to_timedelta(combined_df['Duration'])
combined_df['RouteID'] = combined_df['RouteID'].fillna(0).astype(int)
combined_df['BikeID'] = combined_df['BikeID'].fillna(0).astype(int)
combined_df['BikeName'] = combined_df['BikeName'].str.lower().str.strip().astype(str)

combined_df = combined_df[combined_df['StartDate'] <= combined_df['EndDate']]

combined_df.drop(columns=['StartTime', 'EndTime'], inplace=True)

combined_df.dropna(how='all', inplace=True) 
combined_df.dropna(subset=["StartLongitude", "StartLatitude", "StartHub"], how='all', inplace=True) 
combined_df.dropna(subset=["EndLongitude", "EndLatitude", "EndHub"], how='all', inplace=True)

coord_columns = ['StartLongitude', 'StartLatitude', 'EndLongitude', 'EndLatitude']
# combined_df[coord_columns] = combined_df[coord_columns].round(3)

start_coords = combined_df[['StartLatitude', 'StartLongitude']].drop_duplicates()
end_coords = combined_df[['EndLatitude', 'EndLongitude']].rename(
    columns={'EndLatitude': 'StartLatitude', 'EndLongitude': 'StartLongitude'}
).drop_duplicates()
coords = pd.concat([start_coords, end_coords]).drop_duplicates()
coords['ID'] = range(len(coords))
coords_dict = coords.set_index(['StartLatitude', 'StartLongitude'])['ID'].to_dict()
combined_df['StartID'] = combined_df.apply(
    lambda row: coords_dict[(row['StartLatitude'], row['StartLongitude'])], axis=1
)
combined_df['EndID'] = combined_df.apply(lambda row: coords_dict[(row['EndLatitude'], row['EndLongitude'])], axis=1)


# combined_df['StartID'] = combined_df['StartLatitude'].astype(str) + '##' + combined_df['StartLongitude'].astype(str)
# combined_df['EndID'] = combined_df['EndLatitude'].astype(str) + '##' + combined_df['EndLongitude'].astype(str)

In [162]:
df_for_export = combined_df.copy()

df_for_export.drop(columns=[
    'StartLatitude', 'EndLatitude', 'StartLongitude', 'EndLongitude',
    'StartHub', 'EndHub',
    'PaymentPlan', 'TripType', 'RentalAccessPath', 'MultipleRental',
    'RouteID', 'BikeName' # question: should we leave this in
], inplace=True)

df_for_export['Duration'] = df_for_export['Duration'].dt.total_seconds().astype(int)

df_for_export['StartDate'] = df_for_export['StartDate'].dt.strftime(
    '%Y-%m-%dT%H:%M:%S'
)
df_for_export['EndDate'] = df_for_export['EndDate'].dt.strftime(
    '%Y-%m-%dT%H:%M:%S'
)



In [163]:
from collections import Counter

nodes = {}
# edges = Counter()

for index, row in combined_df.iterrows():
    start_id = row['StartID']
    end_id = row['EndID']
    
    if start_id not in nodes:
        nodes[start_id] = {
            'lat': row['StartLatitude'],
            'lon': row['StartLongitude'],
            'Name': Counter(),
            'Count': 1
        }
    else:
        nodes[start_id]['Count'] += 1
    
    if end_id not in nodes:
        nodes[end_id] = {
            'lat': row['EndLatitude'],
            'lon': row['EndLongitude'],
            'Name': Counter(),
            'Count': 1
        }
    else:
        nodes[end_id]['Count'] += 1
    
    if pd.notna(row['StartHub']):
        nodes[start_id]['Name'][row['StartHub']] += 1
    if pd.notna(row['EndHub']):
        nodes[end_id]['Name'][row['EndHub']] += 1
    
    # edges[(start_id, end_id)] += 1

for node_id, node_data in nodes.items():
    if node_data['Name']:
        most_common_name, _ = node_data['Name'].most_common(1)[0]
        node_data['Name'] = most_common_name
    else:
        node_data['Name'] = None


In [164]:
nodes_df = pd.DataFrame.from_dict(nodes, orient='index')
nodes_df.to_csv("data/Biketown Cleaned Data/hubs.csv", index=False)

df_for_export.to_csv("data/Biketown Cleaned Data/trips.csv", index=False)

In [165]:
tests = [1,2,3,4,5,10,20,100]
og_len = len(nodes_df)
for i in tests:
    length = len(nodes_df[nodes_df['Count'] > i])
    
    print(
        f"{i}: {length / og_len:.0%} ||| {length} / {og_len}"
    )

1: 86% ||| 3939 / 4587
2: 76% ||| 3472 / 4587
3: 71% ||| 3277 / 4587
4: 67% ||| 3056 / 4587
5: 64% ||| 2917 / 4587
10: 54% ||| 2480 / 4587
20: 46% ||| 2088 / 4587
100: 27% ||| 1219 / 4587


In [179]:
nodes_df.reset_index().rename(columns={'index': 'id'})

,id,lat,lon,Name,Count
0,0,45.529760,-122.696643,NW 22nd at Lovejoy,9800
1,52,45.525079,-122.677556,NW Broadway at Everett,18816
2,1,45.522941,-122.688602,None,494
3,5,45.522479,-122.673297,SW 3rd at Ankeny,25054
4,2,45.521622,-122.682415,None,348
...,...,...,...,...,...
4582,4147,45.444012,-122.641277,None,2
4583,4148,45.445517,-122.641659,None,3
4584,4149,45.511527,-122.559101,None,2
4585,4150,45.505369,-122.600286,None,2


In [177]:
nodes

{0: {'lat': 45.52976024,
  'lon': -122.6966429,
  'Name': 'NW 22nd at Lovejoy',
  'Count': 9800},
 52: {'lat': 45.52507854,
  'lon': -122.6775563,
  'Name': 'NW Broadway at Everett',
  'Count': 18816},
 1: {'lat': 45.5229406, 'lon': -122.6886019, 'Name': None, 'Count': 494},
 5: {'lat': 45.52247868,
  'lon': -122.6732969,
  'Name': 'SW 3rd at Ankeny',
  'Count': 25054},
 2: {'lat': 45.5216215, 'lon': -122.6824147, 'Name': None, 'Count': 348},
 84: {'lat': 45.52196048,
  'lon': -122.6896772,
  'Name': 'SW Morrison at 18th',
  'Count': 23204},
 3: {'lat': 45.51928859,
  'lon': -122.6792488,
  'Name': 'SW Morrison at Pioneer Courthouse Sq.',
  'Count': 19758},
 78: {'lat': 45.53080401,
  'lon': -122.6844227,
  'Name': 'NW 13th at Marshall',
  'Count': 31778},
 4: {'lat': 45.531477, 'lon': -122.6844821, 'Name': None, 'Count': 1994},
 127: {'lat': 45.5360712, 'lon': -122.6852715, 'Name': None, 'Count': 7033},
 87: {'lat': 45.52175423,
  'lon': -122.6810794,
  'Name': 'SW 10th at Stark',
  '

In [155]:
df_for_export[(df_for_export['StartID'].isin(nodes_df[nodes_df['Count'] > 100].index)) & (
    df_for_export['EndID'].isin(nodes_df[nodes_df['Count'] > 100].index))]

,StartDate,EndDate,BikeID,DistanceMiles,Duration,StartID,EndID
0,2017-10-01T00:05:00,2017-10-01T00:18:00,6563,1.51,784,0,50
1,2017-10-01T00:17:00,2017-10-01T00:22:00,6273,0.78,311,1,5
2,2017-10-01T00:18:00,2017-10-01T00:22:00,7280,0.43,238,2,77
3,2017-10-01T00:21:00,2017-10-01T00:33:00,6274,1.02,690,3,4
4,2017-10-01T00:22:00,2017-10-01T00:36:00,7112,0.93,820,4,118
...,...,...,...,...,...,...,...
1034816,2017-09-30T23:49:00,2017-09-30T23:53:00,7257,0.61,242,0,91
1034817,2017-09-30T23:50:00,2017-09-30T23:58:00,7385,0.75,448,142,21
1034818,2017-09-30T23:52:00,2017-10-01T00:03:00,6583,1.25,701,251,12
1034819,2017-09-30T23:52:00,2017-09-30T23:58:00,6402,0.81,354,19,9


In [93]:
df_for_export.sample(20).to_clipboard()

In [13]:
df.groupby("StartID").filter(lambda x: x['RouteID'].count() > 100).groupby("StartID").count()

,RouteID,PaymentPlan,StartHub,StartLatitude,StartLongitude,StartDate,StartTime,EndHub,EndLatitude,EndLongitude,EndDate,EndTime,TripType,BikeID,BikeName,DistanceMiles,Duration,RentalAccessPath,MultipleRental,EndID
StartID,,,,,,,,,,,,,,,,,,,,
45.4642942##-122.6638457,143,143,0,143,143,143,143,96,143,143,143,143,0,143,143,143,141,143,143,143
45.4643284##-122.6530114,131,131,0,131,131,131,131,93,131,131,131,131,0,131,131,131,131,131,131,131
45.4913187##-122.6486511,197,197,0,197,197,197,197,118,197,197,197,197,0,197,196,197,197,197,197,197
45.493337##-122.623397,254,254,254,254,254,254,254,174,254,254,254,254,0,254,253,254,254,254,254,254
45.493341##-122.635762,335,335,335,335,335,335,335,238,335,335,335,335,0,335,334,335,335,335,335,335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45.56274555##-122.6480594,897,897,897,897,897,897,897,660,897,897,897,897,2,897,897,897,893,897,897,897
45.56282546##-122.6747467,3480,3480,3480,3480,3480,3480,3480,2579,3480,3480,3479,3479,4,3480,3479,3480,3465,3480,3480,3480
45.5644626##-122.6306698,125,125,0,125,125,125,125,95,125,125,125,125,1,125,124,125,125,125,125,125


,StartDate,BikeID,DistanceMiles,Duration,StartID,EndID
0,2017-10-01T00:05:00,6563,1.51,784,0,50
1,2017-10-01T00:17:00,6273,0.78,311,1,5
2,2017-10-01T00:18:00,7280,0.43,238,2,77
3,2017-10-01T00:21:00,6274,1.02,690,3,4
4,2017-10-01T00:22:00,7112,0.93,820,4,118
...,...,...,...,...,...,...
1034816,2017-09-30T23:49:00,7257,0.61,242,0,91
1034817,2017-09-30T23:50:00,7385,0.75,448,142,21
1034818,2017-09-30T23:52:00,6583,1.25,701,251,12
1034819,2017-09-30T23:52:00,6402,0.81,354,19,9


In [88]:
combined_df.groupby(['BikeID','BikeName']).count()

RouteID  PaymentPlan  StartHub  StartLatitude  \
BikeID  BikeName                                                               
5986.0  0549 biketown             1180         1180       889           1093   
        0549 pbot xm biketown        4            4         4              4   
5987.0  0487 biketown             1250         1250       836           1130   
        0487 mss biketown            9            9         9              9   
5988.0  0696 biketown             1338         1338       960           1170   
...                                ...          ...       ...            ...   
34802.0 service controller 4         1            1         0              1   
35477.0 0315 biketown              113          113        88            113   
        0315 mss biketown            1            1         1              1   
35537.0 0881 biketown               82           82        49             82   
        0881 xc biketown             4            4         2              4   

                               StartLongitude  StartDate  StartTime  EndHub  \
BikeID  BikeName                                                              
5986.0  0549 biketown                    1093       1180       1180     900   
        0549 pbot xm biketown               4          4          4       4   
5987.0  0487 biketown                    1130       1250       1250     891   
        0487 mss biketown                   9          9          9       8   
5988.0  0696 biketown                    1170       1338       1338     979   
...                                       ...        ...        ...     ...   
34802.0 service controller 4                1          1          1       0   
35477.0 0315 biketown                     113        113        113      76   
        0315 mss biketown                   1          1          1       0   
35537.0 0881 biketown                      82         82         82      46   
        0881 xc biketown                    4          4          4       0   

                               EndLatitude  EndLongitude  ...  TripType  \
BikeID  BikeName                                          ...             
5986.0  0549 biketown                 1093          1093  ...         2   
        0549 pbot xm biketown            4             4  ...         0   
5987.0  0487 biketown                 1130          1130  ...         1   
        0487 mss biketown                9             9  ...         0   
5988.0  0696 biketown                 1170          1170  ...         2   
...                                    ...           ...  ...       ...   
34802.0 service controller 4             1             1  ...         0   
35477.0 0315 biketown                  113           113  ...         0   
        0315 mss biketown                1             1  ...         0   
35537.0 0881 biketown                   82            82  ...         0   
        0881 xc biketown                 4             4  ...         1   

                               Distance_Miles  Duration  RentalAccessPath  \
BikeID  BikeName                                                            
5986.0  0549 biketown                    1048      1177              1180   
        0549 pbot xm biketown               0         4                 4   
5987.0  0487 biketown                    1075      1239              1250   
        0487 mss biketown                   0         9                 9   
5988.0  0696 biketown                    1115      1337              1338   
...                                       ...       ...               ...   
34802.0 service controller 4                1         1                 1   
35477.0 0315 biketown                      13       113               113   
        0315 mss biketown                   1         1                 1   
35537.0 0881 biketown                      32        82                82   
        0881 xc biketown                    0         4       

In [88]:
combined_df.groupby(['BikeID','BikeName']).count()

RouteID  PaymentPlan  StartHub  StartLatitude  \
BikeID  BikeName                                                               
5986.0  0549 biketown             1180         1180       889           1093   
        0549 pbot xm biketown        4            4         4              4   
5987.0  0487 biketown             1250         1250       836           1130   
        0487 mss biketown            9            9         9              9   
5988.0  0696 biketown             1338         1338       960           1170   
...                                ...          ...       ...            ...   
34802.0 service controller 4         1            1         0              1   
35477.0 0315 biketown              113          113        88            113   
        0315 mss biketown            1            1         1              1   
35537.0 0881 biketown               82           82        49             82   
        0881 xc biketown             4            4         2              4   

                               StartLongitude  StartDate  StartTime  EndHub  \
BikeID  BikeName                                                              
5986.0  0549 biketown                    1093       1180       1180     900   
        0549 pbot xm biketown               4          4          4       4   
5987.0  0487 biketown                    1130       1250       1250     891   
        0487 mss biketown                   9          9          9       8   
5988.0  0696 biketown                    1170       1338       1338     979   
...                                       ...        ...        ...     ...   
34802.0 service controller 4                1          1          1       0   
35477.0 0315 biketown                     113        113        113      76   
        0315 mss biketown                   1          1          1       0   
35537.0 0881 biketown                      82         82         82      46   
        0881 xc biketown                    4          4          4       0   

                               EndLatitude  EndLongitude  ...  TripType  \
BikeID  BikeName                                          ...             
5986.0  0549 biketown                 1093          1093  ...         2   
        0549 pbot xm biketown            4             4  ...         0   
5987.0  0487 biketown                 1130          1130  ...         1   
        0487 mss biketown                9             9  ...         0   
5988.0  0696 biketown                 1170          1170  ...         2   
...                                    ...           ...  ...       ...   
34802.0 service controller 4             1             1  ...         0   
35477.0 0315 biketown                  113           113  ...         0   
        0315 mss biketown                1             1  ...         0   
35537.0 0881 biketown                   82            82  ...         0   
        0881 xc biketown                 4             4  ...         1   

                               Distance_Miles  Duration  RentalAccessPath  \
BikeID  BikeName                                                            
5986.0  0549 biketown                    1048      1177              1180   
        0549 pbot xm biketown               0         4                 4   
5987.0  0487 biketown                    1075      1239              1250   
        0487 mss biketown                   0         9                 9   
5988.0  0696 biketown                    1115      1337              1338   
...                                       ...       ...               ...   
34802.0 service controller 4                1         1                 1   
35477.0 0315 biketown                      13       113               113   
        0315 mss biketown                   1         1                 1   
35537.0 0881 biketown                      32        82                82   
        0881 xc biketown                    0         4       

In [97]:
combined_df[combined_df['BikeName'] == "0657 biketown"]

,RouteID,PaymentPlan,StartHub,StartLatitude,StartLongitude,StartDate,StartTime,EndHub,EndLatitude,EndLongitude,...,BikeName,Distance_Miles,Duration,RentalAccessPath,MultipleRental,Start_Latitude,Start_Longitude,End_Latitude,End_Longitude,Distance_Miles_
3407,4742287.0,Subscriber,SE Taylor at Chavez,45.514975,-122.623157,10/5/2017,11:27,NaN,45.512882,-122.622926,...,0657 biketown,0.20,0:06:06,keypad,False,NaN,NaN,NaN,NaN,NaN
3439,4742643.0,Subscriber,NaN,45.512882,-122.622926,10/5/2017,11:52,NaN,45.516457,-122.629573,...,0657 biketown,0.53,0:08:28,keypad,False,NaN,NaN,NaN,NaN,NaN
3454,4742787.0,Subscriber,NaN,45.516457,-122.629573,10/5/2017,12:04,NaN,45.516581,-122.621751,...,0657 biketown,0.46,1:14:05,keypad,False,NaN,NaN,NaN,NaN,NaN
8896,4802039.0,Subscriber,NaN,45.516581,-122.621751,10/10/2017,17:30,SE Taylor at Chavez,45.514975,-122.623157,...,0657 biketown,0.13,0:02:34,keypad,False,NaN,NaN,NaN,NaN,NaN
9604,4814953.0,Subscriber,SE Taylor at Chavez,45.514975,-122.623157,10/11/2017,20:33,NaN,45.512049,-122.622839,...,0657 biketown,0.22,0:03:29,keypad,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287150,4599411.0,Casual,SW 5th at Morrison,45.518895,-122.677406,9/24/2017,14:27,SW Salmon at Waterfront Park,45.515568,-122.673892,...,0657 biketown,2.38,0:57:32,keypad,False,NaN,NaN,NaN,NaN,NaN
1287693,4602409.0,Casual,SW Salmon at Waterfront Park,45.515568,-122.673892,9/24/2017,19:34,SW Salmon at Waterfront Park,45.515568,-122.673892,...,0657 biketown,1.54,0:24:17,keypad,False,NaN,NaN,NaN,NaN,NaN
1287761,4602701.0,Casual,SW Salmon at Waterfront Park,45.515568,-122.673892,9/24/2017,20:31,SW Salmon at Waterfront Park,45.515568,-122.673892,...,0657 biketown,6.58,1:23:05,keypad,False,NaN,NaN,NaN,NaN,NaN
1288697,4617651.0,Subscriber,SW River at Montgomery,45.509103,-122.673517,9/25/2017,23:17,SW 3rd at Morrison,45.518039,-122.675582,...,0657 biketown,0.88,0:27:12,keypad,False,NaN,NaN,NaN,NaN,NaN


In [96]:
combined_df.groupby(['BikeName'])['BikeID'].nunique()

BikeName
0001 biketown       1
0001 xm biketown    1
0002 biketown       1
0003 biketown       1
0003 xm biketown    1
                   ..
xc 0424 biketown    1
xc 0603 biketown    1
xc 0610 biketown    1
xc 0655 biketown    1
xm 0683 biketown    1
Name: BikeID, Length: 2298, dtype: int64

In [95]:
combined_df.groupby(['BikeID'])['BikeName'].nunique()

BikeID
5986.0     2
5987.0     2
5988.0     3
5989.0     2
5990.0     1
          ..
30041.0    2
33605.0    1
34802.0    1
35477.0    2
35537.0    2
Name: BikeName, Length: 1044, dtype: int64

In [68]:
combined_df[combined_df['BikeName'].astype(str) == "0006 biketown"]

,RouteID,PaymentPlan,StartHub,StartLatitude,StartLongitude,StartDate,StartTime,EndHub,EndLatitude,EndLongitude,...,BikeName,Distance_Miles,Duration,RentalAccessPath,MultipleRental,Start_Latitude,Start_Longitude,End_Latitude,End_Longitude,Distance_Miles_


In [84]:
combined_df['BikeName'].str.lower().str.strip() == "0006 biketown"

0          False
1          False
2          False
3          False
4          False
           ...  
1294442    False
1294443    False
1294444    False
1294445    False
1294446    False
Name: BikeName, Length: 1294447, dtype: bool

In [130]:
unique_strings = set()

for string in combined_df['BikeName']:
    for substr in str(string).split(" "):
        try:
            int(substr)
        except ValueError:
            unique_strings.add(substr)

In [131]:
unique_strings

{'',
 '(bike',
 '(lost',
 '(new)',
 '(quarantine-do',
 '-',
 '--',
 '-quarantine',
 '0253mss',
 '50th',
 'ability',
 'air',
 'aix',
 'alter',
 'american',
 'ascend',
 'at',
 'betrue',
 'bike',
 'bike,',
 'biketown',
 'biketown,',
 'black',
 'blazers',
 'broken',
 'bunyan',
 'but',
 'challenge',
 'city',
 'colorful',
 'controller',
 'custody',
 'cycling',
 'damaged)',
 'design',
 "explorin'",
 'explorinâ€™',
 'explorin’',
 'explorin‰ûª',
 'for',
 'found',
 'hbo',
 'heritage',
 'history',
 'hq',
 'in',
 'latinx',
 'lck',
 'life',
 'lightbike',
 'likely',
 'lost',
 'lst',
 'max',
 'month',
 'mss',
 'n',
 'nan',
 'native',
 'ne',
 'nike',
 'not',
 'nw',
 'on',
 'out',
 'parts',
 'paul',
 'pbot',
 'pdx',
 'police',
 'portland',
 'possible',
 'prt',
 'qrt',
 'quarantine',
 'recon',
 'reconnw',
 'rewrap',
 'rfs',
 'rider',
 'rip',
 'rma',
 'rmad',
 'rvr',
 'safari',
 'se',
 'service',
 'shop',
 'sneaker',
 'stl',
 'stl/lck',
 'stolen',
 'stolen)',
 'sw',
 'trainer',
 'veterans',
 'waiting',
 